# Projeto Final - Modelos Preditivos - Dataset do Censo

## Grupo:
- Lucas Natan Correia Couri - lncc2@cin.ufpe.br
- Mariama Celi Serafim de Oliveira - mcso@cin.ufpe.br
- Laianna Lana Virginio da Silva - llvs2@cin.ufpe.br
- Priscilla Amarante de Lima - pal4@cin.ufpe.br
- Liviany Reis Rodrigues - lrr@cin.ufpe.br

# Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import kruskal

from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

from numpy.random import randn
from numpy.random import seed

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
#Determinação da SEED utilizada no projeto
SEED = 6138

# 1. Entendimento do Negócio

O objetivo do problema é determinar se uma pessoa ganha mais ou menos de 50 mil dólares. Serão utilizadas as informações do censo americano (14 features) a fim de gerar os modelos.

# 2. Compreensão dos Dados

*   Descrição da base: https://archive.ics.uci.edu/ml/datasets/census+income

Número de Instâncias:
* Dados de Treino: 32561
* Dados de Teste: 16281

Valores Ausentes:
* Foram substituídos por " ?"

Número de Atributos: 14
* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

Distribuição da Classe:
* '>50K' , '<=50K'.
    *   '>50K'  : 23.93% / 24.78% (without ' ?')
    *   '<=50K' : 76.07% / 75.22% (without ' ?')


# 3. Preparação dos Dados

## Carregando a Base de Dados

In [ ]:
columns_name = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class']
df = pd.read_csv("Dados/adult.data", names = columns_name, index_col = False)
df.head(1)

## Análise Exploratória de Dados

Nesta seção, serão analisados a distribuição e característica dos atributos, valores faltantes, possíveis outliers e nível de separatibilidade. 

### Tipos dos dados

In [ ]:
df.dtypes

Mudando os atributos para seus tipos corretos.

In [ ]:
df['workclass'] = df['workclass'].astype('category')
df['education'] = df['education'].astype('category')
df['marital-status'] = df['marital-status'].astype('category')
df['occupation'] = df['occupation'].astype('category')
df['relationship'] = df['relationship'].astype('category')
df['race'] = df['race'].astype('category')
df['sex'] = df['sex'].astype('category')
df['native-country'] = df['native-country'].astype('category')
df['class'] = df['class'].astype('category')
df.dtypes

### Descrição dos dados

Sumário dos dados contínuos.

In [ ]:
df.describe()

### Dados Duplicados

Checando dados duplicados. Ao final, verificamos que não há linhas duplicadas.

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df[df.duplicated()]

### Preenchendo Dados Faltantes

In [ ]:
def tratamento_faltantes(df, columns_name):
    ## Printa os atributos com dados faltantes (" ?")
    for coluna in columns_name:
        if len(df[df[coluna] == " ?"]) > 0:
            print(coluna)
            print(len(df[df[coluna] == " ?"]))
    
    ## Tratamento dos dados faltantes:
    atr_faltantes = ["workclass", "occupation", "native-country"]
    for atr in atr_faltantes:
        categorias_atr = df.groupby(atr).sum().index.tolist()
        label_encoder = preprocessing.LabelEncoder()
        label_encoder.fit(categorias_atr)
        df[atr] = df[atr].replace(" ?", np.nan)
        df[atr] = df[atr].interpolate(method = 'pad')

Tratamento dos dados faltantes: 
1. Transforma para numérico (LabelEnconder) 
2. Substitui " ?" por NaN (replace)
3. Utiliza a frequência dos vizinhos mais próximos para estimar um valor para NaN (interpolate)

In [ ]:
tratamento_faltantes(df, columns_name)

### Checando Outliers

In [ ]:
df['hours-per-week'].plot.box()

In [ ]:
df['hours-per-week'].hist()

In [ ]:
df['capital-gain'].plot.box()

In [ ]:
df['capital-gain'].hist()

In [ ]:
df['capital-loss'].plot.box()

In [ ]:
df['capital-loss'].hist()

### Colunas Redundantes

In [ ]:
df.head(1)

"education" e "education-num" significam a mesma coisa. Vamos utilizar "education-num" e dropar "education", já que "education-num" é a codificação ordinal de "education"

In [ ]:
df['education'].value_counts()

In [ ]:
df['education-num'].value_counts()

### Frequência das Variáveis Categóricas (Value_Counts)

In [ ]:
df['workclass'].value_counts()

In [ ]:
df['education'].value_counts()

In [ ]:
df['marital-status'].value_counts()

In [ ]:
df['occupation'].value_counts()

In [ ]:
df['relationship'].value_counts()

In [ ]:
df['race'].value_counts()

In [ ]:
df['sex'].value_counts()

In [ ]:
df['native-country'].value_counts()

In [ ]:
df['class'].value_counts()

No dataset de treino há apenas uma observação como " Holand-Netherlands", diante do tamanho do dataset (mais de 30mil linhas) optou-se por remover essa única linha com native-country=" Holand-Netherlands" de forma a evitar problemas de ausência do valor no dataset de teste.

In [ ]:
df[df['native-country'] != " Holand-Netherlands"]['native-country'].value_counts()

## Carregando e Processando o Conjunto de Teste

In [ ]:
df_test = pd.read_csv("Dados/adult.test", names = columns_name, index_col = False, skiprows = 1)
df_test.head(1)

### Preenchendo Dados Faltantes

Realizando o mesmo procedimento de input utilizado no conjunto de treinamento.

In [ ]:
tratamento_faltantes(df_test, columns_name)

## Codificação das Variáveis Categóricas

Realizaremos a codificação One Hot Encoding, dado que os modelos a serem utilizados necessitam de entradas numérica. Esse tipo de codifição é indicado para variáveis categóricas nominais, pois atribui distâncias uniformes às categorias.

In [ ]:
def onehot_encoder(df):

    colunas_cat = ["workclass", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
    
    for coluna in colunas_cat:

        #print(coluna)
        df_coluna = pd.get_dummies(df[coluna], prefix=coluna)
        df = df.join(df_coluna)
    
    return df

In [ ]:
df = onehot_encoder(df)
df.head(1)

In [ ]:
df_test = onehot_encoder(df_test)
df_test.head(1)

Verificamos que no conjunto de teste não havia observações com o valor 'Holand-Netherlands' em 'native-country', fato que causava inconsistência com o conjunto de treinamento ao realizar o Hot Enconding. A fim de resolver esse problema, adicionamos manualmente a coluna 'native-country_ Holand-Netherlands'.

In [ ]:
coluna = 'native-country_ Holand-Netherlands'
#df[coluna]
df_test[coluna] = 0

## Normalizando Variáveis Contínuas

Indica-se normalização uma vez que alguns modelos que serão utilizados são baseados em distância. Caso haja escala de distância muito distinta, pode ocorrer enviesamento em algumas features e consequentemente comprometimento na performance do classificador.

In [ ]:
normalize = MinMaxScaler()

In [ ]:
colunas = ["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week", "education-num"]

df[colunas] = normalize.fit_transform(df[colunas])
df_test[colunas] = normalize.fit_transform(df_test[colunas])

In [ ]:
df[colunas].describe()

## Dividindo o Conjuntos de Dados

In [ ]:
colunas_drop = ["class", "education", "workclass", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]

X_train = df.drop(colunas_drop, axis = 1).to_numpy()
y_train = df["class"].values
X_test = df_test.drop(colunas_drop, axis = 1).to_numpy()
y_test = df_test["class"].values

Codificando as classes para valores numéricos:
- '<=50K' :  0
- '>50K'  : 1 

In [ ]:
label_encoder = preprocessing.LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

## Plot do conjunto de Treino (PCA)

In [ ]:
pca = PCA(2)
X_pca = pca.fit_transform(preprocessing.minmax_scale(X_train))

fig, ax = plt.subplots(figsize=(15,10))
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=y_train)

legend1 = ax.legend(*scatter.legend_elements(),
                    loc="lower left", title="Classes")
ax.add_artist(legend1)
plt.show()

# 4. Modelagem

Definindo tamanho do folder no cross validation.

In [ ]:
cv = model_selection.StratifiedKFold(n_splits=10)

## KNN

Para avaliação do KNN foram considerados o número de vizinhos (n_neighbors) e o tipo de distância (metric).

In [ ]:
def val_knn(X_train, y_train, parameters, cv, SEED):
    
    knn = KNeighborsClassifier()
    
    search = RandomizedSearchCV(knn,
                                parameters,
                                n_iter = 50,
                                scoring = 'accuracy',
                                n_jobs = -1,
                                cv = cv,
                                random_state = SEED)
    
    result_knn = search.fit(X_train, y_train)
    
    print('=========Resultados do Random Search para o KNN==========')
    print(f'Melhor Score: {result_knn.best_score_}')
    print(f'Melhores Hiperparâmetros: {result_knn.best_params_}')
    
    return result_knn

In [ ]:
#Definindo parâmetros para avaliação
parameters = dict()
parameters['n_neighbors'] = range(1, 100)
parameters['metric'] = ['euclidean','manhattan']

# result_knn = val_knn(X_train, y_train, parameters, cv, SEED)

**Melhores parâmetros para KNN:**
- n_neighbors: 23
- metric': 'manhattan'

## Árvore de Decisão Simples

Para avaliação da árvore de decisão foram considerados a profundidade da árvore (max_depth), o critério de divisão (criteon), número mínimo de amostra para realizar divisão (min_samples_split), mínimo de amostras necessárias para ser uma folha (min_samples_leaf).

In [ ]:
def val_tree(X_train, y_train, parameters, cv, SEED):
    
    decisionTree = DecisionTreeClassifier()
    
    search = RandomizedSearchCV(decisionTree,
                                parameters,
                                n_iter = 50,
                                scoring = 'accuracy',
                                n_jobs = -1,
                                cv = cv,
                                random_state = SEED)
    
    result_tree = search.fit(X_train, y_train)
    
    print('=========Resultados do Random Search para a Árvore de Decisão Simples==========')
    print(f'Melhor Score: {result_tree.best_score_}')
    print(f'Melhores Hiperparâmetros: {result_tree.best_params_}')
    
    return result_tree

In [ ]:
#Definindo parâmetros para avaliação
parameters = dict()
parameters['criterion'] = ['gini', 'entropy']
parameters['max_depth'] = range(1,30)
parameters['min_samples_split'] = range(1,20)
parameters['min_samples_leaf'] = range(1,10)

# result_tree = val_tree(X_train, y_train, parameters, cv, SEED)

**Melhores parâmetros para Árvore de Decisão:**

- criterion: 'gini'
- max_depth: 11
- min_samples_split: 15
- min_samples_leaf: 2

## Random Forest

Para avaliação do Random Forest foram considerados o número de árvores da floresta (n_estimators), o critério de divisão (criteon), número mínimo de amostra para realizar divisão (min_samples_split), mínimo de amostras necessárias para ser uma folha (min_samples_leaf) e número máximo de features (max_features) .

In [ ]:
def val_rf(X_train, y_train, parameters, cv, SEED):

    rf = RandomForestClassifier(random_state = SEED)

    search = GridSearchCV(rf,
                          parameters,
                          scoring = "accuracy",
                          n_jobs = -1,
                          cv = cv)

    result_rf = search.fit(X_train, y_train)
    
    print('=========Resultados do Grid Search para Random Forest==========')
    print(f'Melhor Score: {result_rf.best_score_}')
    print(f'Melhores Hiperparâmetros: {result_rf.best_params_}')

    return result_rf


### Tentativa 1 (18min)

In [ ]:
parameters = dict()
parameters['n_estimators'] = range(10, 301, 20)
parameters['criterion'] = ["gini", "entropy"]
parameters['max_features'] = ["auto", "sqrt", "log2"]

#result_rf_1 = val_rf(X_train, y_train, parameters, cv, SEED)

### Tentativa 2 (47min)

In [ ]:
parameters = dict()
parameters['n_estimators'] = range(300, 451, 10)
parameters['criterion'] = ["gini", "entropy"]
parameters['max_features'] = ["auto", "sqrt", "log2"]

#result_rf_2 = search_rf(parameters, cv, X_train, y_train, SEED)

### Tentativa 3 (17min)

In [ ]:
parameters = dict()
parameters['n_estimators'] = range(408, 413, 1)
parameters['criterion'] = ["gini", "entropy"]
parameters['max_features'] = ["auto", "sqrt", "log2"]

#result_rf_3 = search_rf(parameters, cv, X_train, y_train, SEED)

### Tentativa 4

In [ ]:
parameters = dict()
parameters['n_estimators'] = [190, 440, 412]
parameters['criterion'] = ["gini", "entropy"]
parameters['max_features'] = ["auto", "sqrt", "log2"]
parameters['min_samples_leaf'] = [1, 4]
parameters['min_samples_split'] = [2, 10]
#parameters['max_depth'] = [10, 100, None]#[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]

#result_rf_4 = search_rf(parameters, cv, X_train, y_train, SEED)

**Melhores parâmetros para Random Forest:**

- n_estimators: 440
- criterion: "gini"
- max_features: "auto"
- min_samples_split: 4
- min_samples_leaf: 10

## Rede Neural MLP

Para avaliação da Rede Neural MLP foram considerados números de camadas escondidas (hidden_layer_sizes), a função de ativação (activation), algorítmos de treinamento (solver), tipo de taxa de aprendizado (learning_rate). 

In [ ]:
def val_mlp(X_train, y_train, parameters, cv, SEED, search_type = 0):

    mlp = MLPClassifier(random_state = SEED)
    
    if search_type == 0:
        search = GridSearchCV(mlp,
                            parameters,
                            scoring = "accuracy",
                            n_jobs = -1,
                            cv = cv)
    else:
        search = RandomizedSearchCV(mlp,
                                    parameters,
                                    n_iter = 50,
                                    scoring = "accuracy",
                                    n_jobs = -1,                            
                                    cv = cv,
                                    random_state = SEED)

    result_mlp = search.fit(X_train, y_train)
    
    print(f'=========Resultados do {"Grid" if (search_type == 0) else "Random"} Search para MLP==========')
    print(f'Melhor Score: {result_mlp.best_score_}')
    print(f'Melhores Hiperparâmetros: {result_mlp.best_params_}')

    return result_mlp

### Tentativa 1 (55 min)

In [ ]:
# Definindo parâmetros do MPL
parameters = dict()
parameters["hidden_layer_sizes"] = [(4,4), (20,15),(50,50), (100,50)] #Testando com 2 camadas
parameters["activation"] = ["logistic", "tanh", "relu", "identity"]
parameters["solver"] = ["lbfgs", "sgd", "adam"]
parameters["learning_rate"] = ["constant", "invscaling", "adaptive"] 

# result_mlp_1 = val_mlp(X_train, y_train, parameters, cv, SEED, 1)

### Tentativa 2

In [ ]:
#Adicionando leaning_rate e retirando algumas opções
parameters = dict()
parameters["hidden_layer_sizes"] = [(4,10,4),(20,10,5),(250, 100, 50)] #Testando com 3 camadas 
parameters["activation"] = ["logistic", "tanh", "relu", "identity"]
parameters["solver"] =  ["lbfgs", "sgd", "adam"]
parameters["learning_rate"] = ["constant", "invscaling", "adaptive"] 

#result_mlp_2 = val_mlp(X_train, y_train, parameters, cv, SEED, 1)

### Tentativa 3

In [ ]:
#Testando melhores resultados da tentativa 1 e 2
parameters = dict()
parameters["hidden_layer_sizes"] = [(20,15), (250, 100, 50)]
parameters["activation"] = ["tanh", "logistic"]
parameters["solver"] = ["lbfgs"]
parameters["learning_rate"] = ["constant", "adaptive"] 

#result_mlp_3 = val_mlp(X_train, y_train, parameters, cv, SEED)

### Tentativa 4

In [ ]:
#Testando camadas mais profundas
parameters = dict()
parameters["hidden_layer_sizes"] = [(5,5,5,5), (10,10,10,10), (50,50,50,50), (10,10,10,10,10)]
parameters["activation"] = ["tanh"]
parameters["solver"] = ["lbfgs"]
parameters["learning_rate"] = ["constant"] 

#result_mlp_4 = val_mlp(X_train, y_train, parameters, cv, SEED)

**Melhores parâmetros para Rede Neural MLP:**

- hidden_layer_sizes: (50, 50, 50, 50)
- activation: 'tanh'
- solver: 'lbfgs'
- learning_rate: 'constant'

## Comitê de Redes Neurais

Para avaliação do Comitê de Redes Neurais foram considerados números de camadas escondidas (hidden_layer_sizes), número de interações (max_iter), quantidade de classificadores no bagging (n_estimators), tamanho da amostra escolhida para treinar os classificadores (max_samples). 

In [ ]:
def val_bgc(X_train, y_train, parameters, cv, SEED, hidden_l, max_iterations):
       
    mlp = MLPClassifier(hidden_layer_sizes = hidden_l,
                        max_iter = max_iterations,
                        random_state = SEED
                        )
    
    search = GridSearchCV(BaggingClassifier(mlp),
                          parameters,
                          scoring = "accuracy",
                          #cv = cv,
                          n_jobs= -1
                          )

    result_bgc = search.fit(X_train, y_train)
   
    print(f'=========Resultados do Grid Search para Bagging Classifier com Redes Neurais MLP==========')
    print(f'Melhor Score: {result_bgc.best_score_}')
    print(f'Melhores Hiperparâmetros: {result_bgc.best_params_}')

    return result_bgc

In [ ]:
parameters = dict()
parameters["n_estimators"] = [10, 20, 30]
parameters["max_samples"] = [0.1, 0.2]

hidden_l = (10, 10)
max_iterations = 200

#result_bgc = val_bgc(X_train, y_train, parameters, cv, SEED, hidden_l, max_iterations)

**Melhores parâmetros para Comitê de Redes Neurais:**

- hidden_layer_sizes: (10, 10)
- max_iter: 200
- n_estimators: 30
- max_samples: 0.2

## Comitê Heterogêneo

Este comitê utiliza quatro estimadores fracos (KNN, árvore de decisão, random forest e rede neural MLP).

In [ ]:
def val_ensemble(X_train, y_train, estimators, cv):
       
    ensemble = VotingClassifier(estimators)
    
    result_ens = model_selection.cross_val_score(ensemble, X_train, y_train, cv = cv)
   
    print(f'=========Resultados do Comitê Heterogêneo com KNN, Árvore de Decisão, Random Forest e MLP==========')
    print(f'Score: {result_ens.mean()}')

    return result_ens

### Tentativa 1

In [ ]:
estimators1 = []

model1 = KNeighborsClassifier()
estimators1.append(('KNN', model1))

model2 = DecisionTreeClassifier(max_depth = 7)
estimators1.append(('Tree', model2))

model3 = RandomForestClassifier()
estimators1.append(('RF', model3))

model4 = MLPClassifier(hidden_layer_sizes = (10,10))
estimators1.append(('MLP', model4))

#result_ens1 = val_ensemble(X_train, y_train, estimators1, cv)

### Tentativa 2

In [ ]:
estimators2 = []

model1 = KNeighborsClassifier()
estimators2.append(('KNN', model1))

model2 = MLPClassifier(hidden_layer_sizes = (10,10))
estimators2.append(('MLP', model2))

#result_ens2 = val_ensemble(X_train, y_train, estimators2, cv)

### Tentativa 3

In [ ]:
estimators3 = []

model1 = DecisionTreeClassifier(max_depth = 6)
estimators3.append(('Tree', model1))

model2 = RandomForestClassifier(n_estimators = 50)
estimators3.append(('RF', model2))

model3 = DecisionTreeClassifier(max_depth = 8)
estimators3.append(('Tree', model3))

model4 = RandomForestClassifier()
estimators3.append(('RF', model4))


#result_ens3 = val_ensemble(X_train, y_train, estimators3, cv)

## Melhores Modelos Encontrados

Definição dos modelos de classificação com as melhores configurações encontradas na seção anterior.

In [ ]:
# criação dos modelos com os melhores parâmetros
knn = KNeighborsClassifier(n_neighbors = 23, metric = 'manhattan')

arvore = DecisionTreeClassifier(min_samples_split = 15, min_samples_leaf = 2, max_depth = 11, criterion = 'gini')

rf = RandomForestClassifier(n_estimators = 440,
                            criterion = "gini",
                            max_features = "auto",
                            min_samples_leaf = 4,
                            min_samples_split = 10,
                            random_state = SEED)

mlp = MLPClassifier(activation = 'tanh',
                    hidden_layer_sizes = (50, 50, 50, 50),
                    learning_rate = 'constant',
                    solver = 'lbfgs')

comite_mlp = BaggingClassifier(MLPClassifier(hidden_layer_sizes = (10, 10),
                                                              max_iter = 10,
                                                              random_state = SEED),
                               n_estimators = 10,
                               max_samples = 10
                              )

comite_h = VotingClassifier(estimators1) #Falta encontrar o melhor estimador

Adicionamos os modelos em uma lista a fim de coletar suas avaliações.

In [ ]:
models = []
models.append(('knn', knn))
models.append(('arvore', arvore))
models.append(('random', rf))
models.append(('mlp', mlp))
models.append(('comite_mlp', comite_mlp))
models.append(('comite_h', comite_h))

### Execução do Modelo Experimental

In [ ]:
#Avaliação de cada modelo nas amotragens estratificas
print('\nDesempenhos médios dos modelos:')

results = []
names = []
for name, model in models:
    
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy')
    
    results.append(cv_results)
    names.append(name)
    
    print(f"{name}: {cv_results.mean()} ({cv_results.std()})")

### Comparação de Modelos

A partir dos valores encontrados na validação cruzada acima, iremos realizar o teste de significânica estatística a fim de comparar os classificadores.

Utilizamos o teste **Kruskal-Wallis**

Passos: 
1. Detectamos se havia distribuições distintas.
2. Caso encontrado uma distribuição distinta, realizamos o teste Kruskal par a par para determinar qual classificador é estatísticamente diferente do outro.

#### Teste de Hipótese Analisando o p-value

In [ ]:
stat, p = stats.kruskal(results[0], results[1], results[2], results[3], results[4], results[5])
print(f"p_value: {p}. Comparison stats: {stat}")

In [ ]:
alpha = 0.05
if p > alpha:
    print('\nMesma distribuição (aceita H0)')
else:
    print('\nDiferentes distribuições (rejeita H0)')

alg = ["KNN        ", "ÁRVORE     ", "RF         ", "MLP        ", "COMITÊ MLP ", "COMITÊ HET."]
print("\nCOMPARAÇÃO:")
for i in range(len(results)):
    for j in range(i+1, len(results)):
        print(f'   {alg[i]} | {alg[j]} -> statistic = {round(stats.kruskal(results[i],results[j]).statistic, 3)},\tp_value = {round(stats.kruskal(results[i],results[j]).pvalue, 4)},\t{"Mesma distribuição (aceita H0)" if (stats.kruskal(results[i],results[j]).pvalue > alpha) else "Diferentes distribuições (rejeita H0)"} ')

## Avaliando o Modelo na Base de Teste

Treinamento dos modelos no conjunto de treino completo (sem divisão de validação).

In [ ]:
knn.fit(X_train, y_train)
arvore.fit(X_train, y_train)
rf.fit(X_train, y_train)
mlp.fit(X_train, y_train)
comite_mlp.fit(X_train, y_train)
comite_h.fit(X_train, y_train)

Predição de cada modelo para a base de teste.

In [ ]:
y_test_prediction_knn = knn.predict(X_test)
y_test_prediction_arvore = arvore.predict(X_test)
y_test_prediction_rf = rf.predict(X_test)
y_test_prediction_mlp = mlp.predict(X_test)
y_test_prediction_comite_mlp = comite_mlp.predict(X_test)
y_test_prediction_comite_h = comite_h.predict(X_test)

# 5. Avaliação

## Apresentação de Resultados

In [ ]:
print("\nAcurácia KNN: Treinamento",  knn.score(X_train, y_train)," Teste" ,knn.score(X_test, y_test))
print("Clasification Report:", classification_report(y_test, y_test_prediction_knn))
print("Confussion Matrix:\n", confusion_matrix(y_test, y_test_prediction_knn))

print("\nAcurácia ÁRVORE: Treinamento",  arvore.score(X_train, y_train)," Teste" , arvore.score(X_test, y_test))
print("Clasification Report:", classification_report(y_test, y_test_prediction_arvore))
print("Confussion Matrix:\n", confusion_matrix(y_test, y_test_prediction_arvore))

print("\nAcurácia RF: Treinamento",  rf.score(X_train, y_train)," Teste" ,rf.score(X_test, y_test))
print("Clasification Report:", classification_report(y_test, y_test_prediction_rf))
print("Confussion Matrix:\n", confusion_matrix(y_test, y_test_prediction_rf))

print("\nAcurácia MLP: Treinamento",  mlp.score(X_train, y_train)," Teste" ,mlp.score(X_test, y_test))
print("Clasification Report:", classification_report(y_test, y_test_prediction_mlp))
print("Confussion Matrix:\n", confusion_matrix(y_test, y_test_prediction_mlp))

print("\nAcurácia COMITÊ MLP: Treinamento",  comite_mlp.score(X_train, y_train)," Teste" ,comite_mlp.score(X_test, y_test))
print("Clasification Report:", classification_report(y_test, y_test_prediction_comite_mlp))
print("Confussion Matrix:\n", confusion_matrix(y_test, y_test_prediction_comite_mlp))

print("\nAcurácia COMITÊ HETEROGÊNEO: Treinamento",  comite_h.score(X_train, y_train)," Teste" ,comite_h.score(X_test, y_test))
print("Clasification Report:", classification_report(y_test, y_test_prediction_comite_h))
print("Confussion Matrix:\n", confusion_matrix(y_test, y_test_prediction_comite_h))

fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# Análise de Features

In [ ]:
#colunas_drop = ["class", "education", "workclass", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]

In [ ]:
model1 = arvore.fit(X_train, y_train)

In [ ]:
text_representation1 = tree.export_text(arvore)
print(text_representation1)

In [ ]:
#plotar a melhor árvore
feature_names = df.drop(colunas_drop, axis = 1).columns

In [ ]:
sorted(zip(feature_names, arvore.feature_importances_), reverse=True,  key=lambda x: x[1])

In [ ]:
fig = plt.figure(figsize=(25,20))

_ = tree.plot_tree(arvore, 
                   feature_names = feature_names,  
                  #  class_names=list(label_encoder.classes_),
                   filled=True,
                   max_depth = 5)

## Propor sugestões de decisões com base nos atributos